# ReforesTree 🌴

We are excited to share with you the ReforesTree dataset! 🎉
We introduce the ReforesTree dataset in hopes of encouraging the fellow machine learning community to take on
the challenge of developing low-cost, scalable, trustworthy and accurate solutions for monitoring, verification and reporting of tropical reforestation inventory. 

#### This is a dataset for the following 6 agroforestry sites
In alphabetical order
1. _Carlos Vera Arteaga_
2. _Carlos Vera Guevara_
3. _Flora Pluas_
4. _Leonor Aspiazu_
5. _Manuel Macias_
6. _Nestor Macias_


## Dataset Components
For each site the data we publish consists of four components free for use:
1. 🛸 Raw drone RGB images _(see wwf_ecuador/"Name of site")_
2. 🌴 Hand measured tree parameters (diameter at breast height, species, biomass, and location) of every tree _(see field_data.csv)_
3. 🔲 Set of bounding boxes of trees for each site cleaned by hand and labeled as banana or not banana _(see annotations/cleaned)_
4. ↔️ Mappings of these bounding boxes with tree labels based on location _(see mappings/final)_


## Tutorial
In this tutorial we go through the steps to recreate (and hopefully improve) the dataset and how to use it. 

Please read our paper [here](https://arxiv.org/abs/2201.11192).
For any questions, please reach out to gyri.reiersen@tum.de or david.dao@inf.eth.ch

## Load packages and modules

In [ ]:
!pip install -Ur requirements.txt

In [1]:
pip install exifread slidingwindow deepforest lxml

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip show deepforest

Name: deepforest
Version: 1.4.1
Summary: Tree crown prediction using deep learning retinanets
Home-page: 
Author: 
Author-email: Ben Weinstein <benweinstein@weecology.org>, Henry Senyondo <henrykironde@gmail.com>, Muhammed Magdy <muhammedmagdy1015@gmail.com>, Om Doiphode <omdoiphode161@gmail.com>, Ethan White <ethan@weecology.org>
License: MIT
Location: /opt/miniconda3/lib/python3.12/site-packages
Requires: aiohttp, aiolimiter, albumentations, docformatter, geopandas, huggingface-hub, matplotlib, nbqa, numpy, opencv-python-headless, pandas, Pillow, progressbar2, pycocotools, pydata-sphinx-theme, Pygments, pytorch-lightning, rasterio, recommonmark, rtree, safetensors, scipy, six, slidingwindow, sphinx, supervision, torch, torchvision, tqdm, xmltodict
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
from rasterio.plot import reshape_as_image
import PIL
from PIL import Image
PIL.Image.MAX_IMAGE_PIXELS = None
import sys
package = os.path.dirname(os.getcwd())
sys.path.append(package)
sys.path.append(package + 'utils')

#import seaborn as sns
#colors = sns.color_palette('tab10')
#mypalette={'NN':colors[0], 'GMN':colors[4], 'OT':colors[1], 'OT on GPS position':colors[1], 'GW':colors[2], 'OT on GPS position + Tree species':colors[3]}
#import matplotlib.pylab as plt
#import tensorflow

import warnings
warnings.filterwarnings('ignore')

In [4]:
from utils.extract_features import *
from utils.deepforest_detection import *
#from utils.visualisation import *
#from utils.plot_folium import *
#from utils.plot_density import *
#from utils.mapping import *

Data Importing

In [7]:
pip install torchgeo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.5/454.5 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 23.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.4/899.4 kB 20.4 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 834.9/834.9 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.0/811.0 kB 21.

In [8]:
from torchgeo.datasets import ReforesTree
ds = ReforesTree(root="data/reforestree/", download=True, checksum=True)

 74%|███████▍  | 5.59G/7.51G [08:22<02:52, 11.1MB/s] 


KeyboardInterrupt: 

## Data Processing

### 🛸 Split raw drone RGB orthomosaics into image tiles
1. **Extract drone informations on each site:**
Based on the raw data we create the dataframe _ortho_data.csv_ that contains all important information on the drone orthomosaics RGB: minimimal and maximal latititude, longitude, width and height (pixels).


2. **Split the orthomosaics into 400x400 tiles:**
To be able to handle the drone data we need to split the large .tif files into tiles that is stored in _data/tiles_. 

In [5]:
directory = "data/wwf_ecuador/RGB Orthomosaics"
save_dir = "data/tiles"

# Extracting the main information for each site 
ortho_data = create_ortho_data(directory, os.path.join(save_dir, 'ortho_data.csv'))

FileNotFoundError: [Errno 2] No such file or directory: 'data/wwf_ecuador/RGB Orthomosaics'

In [ ]:
# Split images into tiles (might takes some minutes)
for file in os.listdir(directory):
    if file.endswith('.tif'):
        # Open image file for reading (binary mode)
        path_to_raster = os.path.join(directory, file)
        name = file.replace('.tif', '')

        tiles_dir = os.path.join(save_dir, name)
        if not os.path.exists(tiles_dir):
            os.makedirs(tiles_dir)        
            split_raster(path_to_raster, base_dir=tiles_dir, patch_size=4000, patch_overlap=0.05)

### 🌴 Rescale drone bounds using field data

The "_field_data.csv_" contains all the trees on the sites. Each row includes information on the tree parameters as well as the locations and calculated aboveground biomass (AGB) and carbon. 

PS: It is worth noting that the column "updated diameter" is being used for DBH. For completeness and transparancy, the comlumn "diameter" is kept and for the trees with missing values (especially cacao) an extrapolated diameter was calculated based on avg. diameter for that species for the year the tree was planted. 

A separate tutorial on how the processing of the field data is TBD.

In [ ]:
field_data = pd.read_csv('data/field_data.csv')
field_data

In [ ]:
# Then to the actual rescaling
list_sites = ['Carlos Vera Arteaga RGB', 'Carlos Vera Guevara RGB', 'Flora Pluas RGB', 'Leonor Aspiazu RGB', 
             'Manuel Macias RGB', 'Nestor Macias RGB']

ortho_data = rescale_bounds(ortho_data, field_data, list_sites)
ortho_data.to_csv(os.path.join(save_dir, 'ortho_data.csv'), index = False)

## 🔲 DeepForest Tree Detection

From tiles to bounding box annotations! For that we use a pretrained and finetuned DeepForest model.

Note reg finetuning: To make sure our model was able to detect the trees in these sites accurately, we finetuned it on some hand-created bounding boxes. 

In [ ]:
# Load model 

model = deepforest.deepforest(saved_model = os.getcwd()+'/data/model/deepforest/final_model_4000_epochs_35.h5')

In [ ]:
# Detect trees per tile and return bounding box annotations for each site
column_names = ['img_path', 'xmin', 'ymin', 'xmax', 'ymax', 'score']

dir = os.path.join(os.getcwd(), 'data')
tiles_dir = os.path.join(dir,'tiles')
ann_dir = os.path.join(dir,'annotations')

if not os.path.exists(ann_dir):
    os.makedirs(ann_dir)

for folder in os.listdir(tiles_dir):
    if not (folder.startswith('.') or folder.startswith('ortho_data.csv')):
        file_path = ann_dir + '/{}_annotations.csv'.format(folder)
        if not os.path.exists(file_path):
            annotations_files = pd.DataFrame(columns = column_names)
            folder_path = os.path.join(tiles_dir,folder)
            
            for file in os.listdir(folder_path):
                if not file.startswith('.'):
                    tile_annotations = get_annotations(os.path.join(folder_path, file), model)
                    annotations_files = pd.concat([annotations_files, tile_annotations])
            annotations_files = annotations_files.reset_index(drop=True)

            annotations_files.to_csv(file_path, index=False)
        print('DeepForest annotations are saved for site {}'.format(folder))


#### Additional tile information per bounding box annotation

For each tree annotation we add tree location, tile position, and site.

In [ ]:
for site_name in list_sites:
    for file in os.listdir(ann_dir):
        if (file == '{}_annotations.csv'.format(site_name)):
            file_path = os.path.join(ann_dir,file)
            df = pd.read_csv(file_path)
            
            if not ('Xmin' in df.columns): # in case you have done this before
                df['img_name'], df['tile_index'], df['tile_xmin'], df['tile_ymin'], df['tile_xmax'], df['tile_ymax'] = zip(*df['img_path'].map(expand_tile_features))
                df[['x', 'y']] = df.apply(lambda x: [get_center(x.xmin,x.xmax), get_center(x.ymin,x.ymax)], axis=1, result_type="expand")

                df['Xmin'] = df.xmin + df.tile_xmin
                df['Ymin'] = df.ymin + df.tile_ymin
                df['Xmax'] = df.xmax + df.tile_xmin
                df['Ymax'] = df.ymax + df.tile_ymin
                df['X'] = df.x + df.tile_xmin
                df['Y'] = df.y + df.tile_ymin

                df[['lon', 'lat']] = df.apply(lambda x: convert_xy_tile_to_lonlat(x.img_name, x.tile_xmin, x.tile_ymin, x.x, x.y, ortho_data), axis=1, result_type="expand")
                df.to_csv(file_path, index = False)
            print('Added info for {}'.format(site_name))
        

In [ ]:
# Merge annotations files of each site into one file
all_annotations = pd.DataFrame()

for file in os.listdir(ann_dir):
    if not (file.startswith('.') or file.startswith('c') or file.startswith('a')):
        file_path = os.path.join(ann_dir, file)
        df = pd.read_csv(file_path)
        all_annotations = pd.concat([all_annotations, df])
all_annotations.to_csv(os.path.join(ann_dir,'all_annotations.csv'))

### 🍌 Cleaned bounding boxes
As part of our work, we have manually cleaned the bounding boxes coming out of deepforest. This was done in two parts; one filtering out bboxes that were not of trees, due to size (e.g. too large area or only of a leaf) or due to wrong detection (of a car or grass). The second part consisted of labelling the trees as either "banana" (easily recognizable looking like a palm tree) or "other". 

Please find the cleaned dataset in "_data/annotations/cleaned_".

In [ ]:
clean_annotations = pd.read_csv('data/annotations/cleaned/clean_annotations.csv')
clean_annotations.columns

## ↔️ Matching of tree label + bounding box

TBD